In [ ]:
# PART 1 - Imports & Configuration

import pandas as pd
import os

# Input source file 
FILE_PATH = r"C:\Users\khali\OneDrive\Pictures\Desktop\Data file for students.xlsx"

# Output file 
OUTPUT_FILE = os.path.join(os.path.dirname(FILE_PATH), "financial_kpis.xlsx")


In [ ]:

# PART 2 - Load and Clean Sheets

# Load all sheets from Excel
dfs = pd.read_excel(FILE_PATH, sheet_name=None)

# Remove unnamed/empty columns
for sheet in dfs.keys():
    dfs[sheet] = dfs[sheet].loc[:, ~dfs[sheet].columns.str.contains("^Unnamed")]

# Extract key sheets
gl_df = dfs["GL"].copy()
chart_df = dfs["Chart of Accounts"].copy()
calendar_df = dfs["Calendar"].copy()
territory_df = dfs["Territory"].copy()

# Convert Date to datetime and add Year column
gl_df["Date"] = pd.to_datetime(gl_df["Date"], errors="coerce")
gl_df["Year"] = gl_df["Date"].dt.year


In [ ]:
# PART 3 - Enrich General Ledger


# Merge GL with Chart of Accounts to get Class, Report, etc.
gl_df = gl_df.merge(chart_df, on="Account_key", how="left")

# Merge GL with Territory to get Region / Country
gl_df = gl_df.merge(territory_df, on="Territory_key", how="left")

# Merge GL with Calendar to get Year, Quarter, Month, Day
gl_df = gl_df.merge(calendar_df, on="Date", how="left")


In [ ]:
# PART 4 - Clean Columns & Compute KPIs

# Fix Year column from merges
if "Year_x" in gl_df.columns:
    gl_df = gl_df.rename(columns={"Year_x": "Year"})
if "Year_y" in gl_df.columns:
    gl_df = gl_df.drop(columns=["Year_y"], errors="ignore")

# Keep the left side of merges
gl_df = gl_df.rename(columns={
    "Report_x": "Report",
    "Class_x": "Class",
    "SubClass_x": "SubClass",
    "SubClass2_x": "SubClass2",
    "Account_x": "Account",
    "SubAccount_x": "SubAccount"
})

# Drop duplicate right-side columns
gl_df = gl_df.drop(columns=[c for c in gl_df.columns if c.endswith("_y")], errors="ignore")

# Normalize text columns to be robust to casing/spacing
for col in ["Report", "Class", "SubClass", "SubClass2", "Account", "SubAccount", "Details"]:
    if col in gl_df.columns:
        gl_df[col] = gl_df[col].astype(str).str.strip()
        gl_df[f"{col}_norm"] = gl_df[col].str.lower().str.strip()
    else:
        gl_df[f"{col}_norm"] = ""

# Debug
print("✅ Columns after cleaning:", gl_df.columns.tolist())


# CAPEX via explicit 'Details' 
CAPEX_DETAILS_PAT = r"\bpurchase of (equipment|intangible|intangibles)\b"

def capex_amount(d):
    """Return CAPEX amount for a yearly slice d (robust)."""
    # 1) Primary: details mention of purchases
    mask_details = d["Details_norm"].str.contains(CAPEX_DETAILS_PAT, regex=True, na=False)

    # 2) Fallback: movements on PPE / Intangible Assets (when booked as purchases in GL)
    #    This is a proxy in case Details are missing. We keep it permissive.
    mask_ppe = d["Account_norm"].isin(["property, plant, & equipment", "intangible assets"])

    # Combine
    mask_capex = mask_details | mask_ppe

    return d.loc[mask_capex, "Amount"].sum()

# ---------------- Indicators ----------------
def compute_indicators(df):
    """Compute yearly financial indicators from the enriched GL dataframe."""
    if "Year" not in df.columns:
        raise KeyError("⚠️ Column 'Year' not found in dataframe.")

    grp = df.groupby("Year")

    out = grp.apply(lambda d: pd.Series({
        # Sales = Trading account
        "Sales(CA)": d.loc[d["Class"] == "Trading account", "Amount"].sum(),

        # OPEX = Operating account 
        "OPEX": d.loc[d["Class"] == "Operating account", "Amount"].sum(),

        # EBITDA = Sales + OPEX 
        "EBITDA": (
            d.loc[d["Class"] == "Trading account", "Amount"].sum()
            + d.loc[d["Class"] == "Operating account", "Amount"].sum()
        ),

        # Interest & Tax 
        "Interest_Tax": d.loc[d["Class"] == "Interest & Tax", "Amount"].sum(),

        # Net Income = Sales + OPEX + Non-operating + Adjusting - Interest & Tax
        "NetIncome": (
            d.loc[d["Class"] == "Trading account", "Amount"].sum()
            + d.loc[d["Class"] == "Operating account", "Amount"].sum()
            + d.loc[d["Class"] == "Non-operating", "Amount"].sum()
            + d.loc[d["Class"] == "Adjusting", "Amount"].sum()
            - d.loc[d["Class"] == "Interest & Tax", "Amount"].sum()
        ),

        # WCR(BFR)
        "WCR(BFR)": (
            d.loc[(d["Report"] == "Balance Sheet") & (d["SubClass2"] == "Current Assets"), "Amount"].sum()
            - d.loc[(d["Report"] == "Balance Sheet") & (d["SubClass2"] == "Current Liabilities"), "Amount"].sum()
        ),

        # CAPEX: robust (Details mention OR Account is PPE/intangibles)
        "CAPEX": capex_amount(d)
    }))

    return out

# Compute Indicators
indicators = compute_indicators(gl_df)
print("✅ Indicators computed:")
print(indicators)

# Breakdown per class (useful for Power BI)
class_per_year = (
    gl_df.groupby(["Year", "Class"])["Amount"]
    .sum()
    .unstack(fill_value=0)
    .sort_index()
)


✅ Columns after cleaning: ['EntryNo', 'Date', 'Territory_key', 'Account_key', 'Details', 'Amount', 'Year', 'Report', 'Class', 'SubClass', 'SubClass2', 'Account', 'SubAccount', 'Country', 'Region', 'Quarter', 'Month', 'Day', 'Report_norm', 'Class_norm', 'SubClass_norm', 'SubClass2_norm', 'Account_norm', 'SubAccount_norm', 'Details_norm']
✅ Indicators computed:
      Sales(CA)     OPEX   EBITDA  Interest_Tax  NetIncome  WCR(BFR)    CAPEX
Year                                                                         
2018    2383246 -1642445   740801       -153422     306844   2438860  -993004
2019    3968546 -2492858  1475688       -219703     439406   3757253 -1161156
2020    5341360 -3819194  1522166       -304486     608972   1169074 -1530264


C:\Users\khali\AppData\Local\Temp\ipykernel_18508\2670222018.py:42: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask_details = d["Details_norm"].str.contains(CAPEX_DETAILS_PAT, regex=True, na=False)
C:\Users\khali\AppData\Local\Temp\ipykernel_18508\2670222018.py:61: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  out = grp.apply(lambda d: pd.Series({


In [ ]:
# PART 5 - Save / Update Excel File 


from openpyxl import load_workbook

def save_full_output(gl_df, class_per_year, indicators):
    """
    Save everything in the Excel file:
    - Enriched GL
    - Class per Year
    - Indicators
    Optimized: adjust column width quickly + format date.
    """

    # Format Date column
    if "Date" in gl_df.columns:
        gl_df["Date"] = pd.to_datetime(gl_df["Date"], errors="coerce").dt.strftime("%Y-%m-%d")

    # Ensure output directory exists
    os.makedirs(os.path.dirname(OUTPUT_FILE), exist_ok=True)

    # Save dataframes into Excel
    with pd.ExcelWriter(OUTPUT_FILE, engine="openpyxl", mode="w") as writer:
        gl_df.to_excel(writer, sheet_name="General_Ledger", index=False)
        class_per_year.to_excel(writer, sheet_name="Class_Per_Year")
        indicators.to_excel(writer, sheet_name="Indicators")

    # Re-open with openpyxl to adjust column widths
    wb = load_workbook(OUTPUT_FILE)
    for sheet_name in wb.sheetnames:
        ws = wb[sheet_name]
        for col in ws.columns:
            max_length = 0
            col_letter = col[0].column_letter
            # Check only first 100 rows for performance
            for cell in col[:100]:
                if cell.value:
                    max_length = max(max_length, len(str(cell.value)))
            ws.column_dimensions[col_letter].width = min(max_length + 2, 40)
    wb.save(OUTPUT_FILE)

    print(f"✅ File created successfully → {OUTPUT_FILE}")


# Save everything
save_full_output(gl_df, class_per_year, indicators)


✅ File created successfully → C:\Users\khali\OneDrive\Pictures\Desktop\financial_kpis.xlsx
